In [5]:
import numpy as np
from PIL import Image
from pathlib import Path

In [6]:
def get_face_size(subj_id):
    assert subj_id >= 0
    assert subj_id <= 22
    
    img = np.array(Image.open(f"Skin_code/data/ICT_3DRFE_mod/S{subj_id:03d}//shader/S{subj_id:03d}_E00_Mask.bmp"))

    col_forehead = [247,42,206]
    col_cheeks = [215,245,151]
    col_chin = [228,149,79]

    mask1 = np.all(img == col_forehead, axis=2)
    mask2 = np.all(img == col_cheeks, axis=2)
    mask3 = np.all(img == col_chin, axis=2)
    mask = mask1 | mask2 | mask3
    img[~mask, :] = [0,0,0]

    nz = np.nonzero(img)
    h = nz[0].max() - nz[0].min()
    w = nz[1].max() - nz[1].min()
    
    return h,w

In [7]:
def create_patches(subj_id):
    ih, iw = get_face_size(subj_id)

    height_px_per_cm = ih/19.0
    width_px_per_cm = iw/14.0

    mask_img = np.array(Image.open(f"Skin_code/data/ICT_3DRFE_mod/S{subj_id:03d}//shader/S{subj_id:03d}_E00_Mask.bmp"))
    skin_img = np.array(Image.open(f"Skin_code/data/ICT_3DRFE_mod/S{subj_id:03d}//shader/diff_texture.bmp"))

    count = 0
    height_step = int( 0.8 * height_px_per_cm) # 20% overlap to allow for a smoother transition to be more likely from the GAN
    width_step = int(0.8 * width_px_per_cm)

    for i in range(0,skin_img.shape[0]-height_step,height_step):
        for j in range(0,skin_img.shape[1]-width_step,width_step):
            if np.all(mask_img[i:i+int(height_px_per_cm),j:j+int(width_px_per_cm)] > 0):
                Path(f"patches/S{subj_id:03d}").mkdir(parents=True, exist_ok=True)
                Image.fromarray(skin_img[i:i+int(height_px_per_cm),j:j+int(width_px_per_cm)]).save(f"patches/S{subj_id:03d}/{count:03d}.bmp")
                count += 1

In [8]:
for subject in range(0,23):
    create_patches(subject)